Extreme fun with cryptography - Play with encryption - Hash, Salt. Playing with steps below.
1. Accept input from a user, and with the input as an argument
2. Create a random 32-byte salt value, use a cryptographically secure random number generator.
3. Hash the input from the user with the salted value with SHA-256.
Store the value and the salt in a file.
4. Print both values.
5. Write another function that changes the stored hash only after first validating that the password is known.  
6. Test

In [ ]:
#Part 1 to 4

import os          # Import os for generating a secure random salt
import hashlib     # Import hashlib for SHA-256 hashing

def hash_with_salt(user_input):
    salt = os.urandom(32)  # Generate a random 32-byte salt using a cryptographically secure generator
    input_bytes = user_input.encode('utf-8')  # Convert the user input string to bytes for hashing
    hash_obj = hashlib.sha256(input_bytes + salt)  # Hash the combination of input bytes and salt using SHA-256
    hash_digest = hash_obj.hexdigest()  # Convert the hash to a hexadecimal string for easy display
    print(f"Hash: {hash_digest}")       # Print the resulting hash value
    print(f"Salt: {salt.hex()}")        # Print the salt as a hexadecimal string

user_input = input("Enter your input: ")  # Prompt the user for input
hash_with_salt(user_input)                # Call the function with the user's input

Enter your input: omer
Hash: 91b0974669e61d735bcb26e7434096d8878fe73042f6a43741c260e83735ad36
Salt: 92e514650625cece2895217e4a39bed9c91f15fbe326d2f5d1579db30878ef13


Learning: I thought that with Hash or Salt, it will actually contain the input but I learned that the output is actually totally different.

I learned:

Salt: The salt is a random sequence of bytes, generated independently of input. It will look like a long string of random hexadecimal characters and will never contain your input.
Hash: The hash is the result of applying the SHA-256 algorithm to the combination of your input and the salt. Hash functions are designed to be one-way and to produce outputs that look random, even if the input is similar. We will not see your input (like "Omer") in the hash output. The hash will also look like a long string of random hexadecimal characters. The hash is a cryptographic transformation of your input and the salt, designed to be irreversible and not reveal the input.


In [ ]:
#Part 5 and 6

import os          # Import os for generating a secure random salt
import hashlib     # Import hashlib for SHA-256 hashing

# Function to create a hash and salt from a password
def create_hash_and_salt(password):
    salt = os.urandom(32)  # Generate a random 32-byte salt
    password_bytes = password.encode('utf-8')  # Convert password to bytes
    hash_obj = hashlib.sha256(password_bytes + salt)  # Hash password + salt
    hash_digest = hash_obj.hexdigest()  # Get hex representation of hash
    return hash_digest, salt  # Return both hash and salt

# Function to verify a password against a stored hash and salt
def verify_password(stored_hash, stored_salt, password_attempt):
    password_bytes = password_attempt.encode('utf-8')  # Convert attempt to bytes
    hash_obj = hashlib.sha256(password_bytes + stored_salt)  # Hash attempt + stored salt
    hash_attempt = hash_obj.hexdigest()  # Get hex representation
    return hash_attempt == stored_hash  # Return True if hashes match

# Function to update the stored hash if the correct password is provided
def update_password(stored_hash, stored_salt, old_password, new_password):
    if verify_password(stored_hash, stored_salt, old_password):  # Check if old password is correct
        new_hash, new_salt = create_hash_and_salt(new_password)  # Create new hash and salt
        print("Password updated successfully!")  # Inform user of success
        return new_hash, new_salt  # Return new hash and salt
    else:
        print("Incorrect password. Cannot update.Do better!!!")  # Inform user of failure
        return stored_hash, stored_salt  # Return old hash and salt unchanged

# --- DEMONSTRATION ---

# Step 1: Set an initial password
initial_password = "mySecret123"  # The original password
stored_hash, stored_salt = create_hash_and_salt(initial_password)  # Store hash and salt

print("Initial hash:", stored_hash)  # Show initial hash
print("Initial salt:", stored_salt.hex())  # Show initial salt

# Step 2: Try to update password with the wrong old password
stored_hash, stored_salt = update_password(stored_hash, stored_salt, "wrongPassword", "newSecret456")

# Step 3: Update password with the correct old password
stored_hash, stored_salt = update_password(stored_hash, stored_salt, "mySecret123", "newSecret456")

print("Updated hash:", stored_hash)  # Show updated hash
print("Updated salt:", stored_salt.hex())  # Show updated salt

Initial hash: deb81474c560ba7844ace9976b9e60ba933b425699a4712aa27457aa21188d49
Initial salt: 62fac3e5c602a11490202db258df1eeb08687ad3882a16ade3e5fe87d7998533
Incorrect password. Cannot update.Do better!!!
Password updated successfully!
Updated hash: 96d44b682604d1e2e4516318a47b1dd3be617530866fbb77e3a868c8622af480
Updated salt: 19ca97a40393540123040aa469ac9b504fa5d6e241a1eaf2ecff4ed0e98fee7c


Part 7

A. Discuss why a salted hash is important when storing sensitive data such as passwords.
B. Explore a python library for Zero Knowledge Proofs and propose how this might improve authentication. (Cite sources)

Answer A:
Salted Hash is critical for storing passwords because:
1. Prevents Rainbow attack by adding uniqueness to the password making it hard to crack or reversible. Rainbow table reverses cryptographic hash functions.
2. If 2 users have same password their hashes will also be same and adding salt makes each unique. Therefore, harder to crack.
3. Salting forces attackers to compute hash for each password individually, increasing the workload on the attacker and making large scale attacks effort intensive. Even if attacker has hash algorhithm, the salt makes each password unique, forcing the attacker to figure it out individually.

Sources:
a. https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html

Answer B:
Zero Knowledge Proofs allow one party to prove to another that they know a value, like a password, without revealing any information about the value itself.

Python Library Example:
a. py_ecc and pyzksnark: Implements elliptic curve operations, which are foundational for many ZKP protocols.
b. pyzksnark: A Python binding for zkSNARKs, a popular ZKP protocol.

Traditional authentication requires the user to send a password or a hash to the server, which can be intercepted or leaked. With ZKP-based authentication, the user can prove they know the password without ever sending it or its hash to the server. This means, even if the communication is intercepted, the attacker learns nothing about the password.The server does not need to store password hashes, reducing the risk if the server is compromised.Replay attacks are prevented, as each proof is unique and cannot be reused.

Sources:
a. https://www.youtube.com/playlist?list=PLoROMvodv4rO1NB9TD4iUZ3qghGEGtqNX


Part 8
Name one AES encryption mode that requires an Initial Vector (IV) and explain why it is considered more secure than other modes.  

One common AES encryption mode that requires an Initialization Vector is Cipher Block Chaining mode.
The reason CBC is more secure is because it prevents pattern decoding or recognition. In Electronic Codebook mode, identical plaintext blocks always produce identical ciphertext blocks, which can reveal patterns in the data. In CBC mode, each plaintext block is stored with the previous ciphertext block before encryption. The first block uses the IV instead of a previous ciphertext block.The IV ensures that even if the same plaintext is encrypted multiple times with the same key, the ciphertext will be different each time (as long as the IV is unique and random). This randomizes the block making it harder to decode. This randomness prevents attackers from deducing information about the plaintext from repeated ciphertext patterns.

Sources:
a. https://nvlpubs.nist.gov/nistpubs/Legacy/SP/nistspecialpublication800-38a.pdf

Part 9

Name one AES encryption mode that does not require an IV.  


One AES encryption mode that does not require an Initialization Vector is Electronic Codebook mode. In ECB mode, each block of plaintext is encrypted independently using the same key, and no IV is used or needed. However, this lack of an IV makes ECB mode less secure, as identical plaintext blocks will always produce identical ciphertext blocks, potentially revealing patterns in the data.
This takes me back to the example discussed in class on the Enigma machine where there was a repitition of a message block, which was decoded and enabled decoding the entire message! Hence CBC is critical.

Sources:
a. https://nvlpubs.nist.gov/nistpubs/Legacy/SP/nistspecialpublication800-38a.pdf